In [1]:
import torch
print(torch.cuda.is_available())  # doit être True

True


In [2]:
import pandas as pd

df = pd.read_csv("/kaggle/input/politixpert-cleaned-data/politixpert_data_cleaned.csv")

# garder uniquement les textes valides
df = df[df["content"].notna()]
df = df[df["content"].str.len() > 100].reset_index(drop=True)

In [3]:
def chunk_text(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap

    return chunks

In [4]:
df.columns

Index(['source', 'date', 'link', 'title', 'content', 'description', 'type',
       'author', 'categories'],
      dtype='object')

In [5]:
chunks = []
metadata = []

for idx, row in df.iterrows():
    text_chunks = chunk_text(row["content"])

    for i, chunk in enumerate(text_chunks):
        chunks.append(chunk)
        metadata.append({
            "article_id": idx,
            "chunk_id": f"{idx}_{i}",
            "link": row["link"],
            "source": row["source"],
            "date": str(row["date"]),
            "title": row["title"],
            "description": row["description"],
            "author": row["author"],
            "type": row["type"],
            "categories": row["categories"]
        })

In [6]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer(
    "intfloat/multilingual-e5-base",
    device="cuda"
)

2025-12-14 14:23:31.471142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765722211.645816      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765722211.696428      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [7]:
embeddings = model.encode(
    chunks,
    batch_size=64,
    show_progress_bar=True,
    normalize_embeddings=True
)

embeddings = np.array(embeddings).astype("float32")
print("Embeddings shape:", embeddings.shape)

Batches:   0%|          | 0/1418 [00:00<?, ?it/s]

Embeddings shape: (90727, 768)


In [8]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 95.0 MB/s eta 0:00:00:00:0100:01


In [9]:
import faiss
print(faiss.get_num_gpus())

0


In [10]:
import faiss
import numpy as np

embeddings = np.array(embeddings).astype("float32")

dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

print("Total vectors:", index.ntotal)

Total vectors: 90727


In [11]:
def semantic_search(query, top_k=15, min_score=0.82):
    # 1. Encodage avec le modèle global 'model'
    q_emb = model.encode([query], normalize_embeddings=True).astype("float32")
    
    # 2. Recherche élargie
    search_k = top_k * 2
    scores, indices = index.search(q_emb, search_k)
    
    results = []
    
    for i in range(search_k):
        score = float(scores[0][i])
        idx = indices[0][i]
        
        # --- FILTRE 1 : SCORE ---
        # Si le score est trop bas, on ignore.
        # Note : 0.80 est prudent. Pour l'espace c'était ~0.81, donc essayez 0.82 si nécessaire.
        if score < min_score:
            continue
            
        # --- FILTRE 2 : LONGUEUR ---
        # On récupère le texte depuis la liste globale 'chunks'
        if idx < len(chunks):
            text_content = chunks[idx]
            
            # Si le texte est trop court (< 50 caractères), on ignore
            if len(text_content) < 50:
                continue

            # Ajout au résultat
            results.append({
                "score": score,
                "text": text_content,
                **metadata[idx] # Utilisation de la liste globale 'metadata'
            })
            
            # Stop si on a assez de résultats
            if len(results) >= top_k:
                break
    
    return results

In [12]:
# Test avec le filtre activé
# Les résultats sur l'espace devraient disparaître si leur score < 0.80 (ou le seuil choisi)
results = semantic_search("ما موقف الأحزاب من قطاع الصحة؟", min_score=0.82)

if not results:
    print("✅ Aucun document trouvé (Filtre réussi !)")
else:
    for r in results:
        print(r["score"], r["source"], r["title"])

0.8473044037818909 حزب التقدم والاشتراكية بلاغ المكتب السياسي لحزب التقدم والاشتراكيه ليوم الخميس غشت
0.8463587164878845 حزب التقدم والاشتراكية المكتب المحلي لحزب التقدم و الاشتراكيه بالمريسه بسلا يستقبل وفدا عن التنسيق النقابي لقطاع الصحه
0.8421285152435303 حزب التقدم والاشتراكية الرفيق رشيد حموني التعديل خضع ل ترضيه الخواطر
0.8414610028266907 حزب التقدم والاشتراكية تصريح المكتب السياسي لحزب التقدم والاشتراكيه
0.8410338163375854 الحركة الشعبية النايب البرلماني محمد اوزين يدخل تنامي موجات الاحتجاج بسبب تردي الاوضاع الصحيه بالمستشفيات العموميه لقبه البرلمان
0.8406890630722046 الحركة الشعبية ديناميه الحركه الشعبيه مستمره وليست موسميه انتخابيه
0.8404852151870728 الاتحاد الاشتراكي للقوات الشعبية الفريق الاشتراكي بمجلس النواب يرد علي رييس الحكومه حول تاهيل قطاع الصحه
0.8399141430854797 التجمع الوطني للأحرار اخنوش الاحرار يراهن علي اصلاح قطاع الصحه عبر كفاءات تتوفر علي حلول واقعيه وعمليه
0.8397669792175293 الحركة الشعبية خلال الموتمر التاسيسي للهييه الطبيه الشعبيه اوزين هاجسنا اخراج الكفاءات

In [13]:
results = semantic_search("ما هو موقف الأحزاب من غزو الفضاء؟")

for r in results:
    print(r["score"], r["source"], r["title"])

In [14]:
import torch
from transformers import pipeline

# Chargement du modèle Qwen (Plus intelligent et performant en Arabe)
model_id = "Qwen/Qwen2.5-1.5B-Instruct"

print(f"Chargement du générateur {model_id}...")

# On utilise float16 pour que ça soit rapide et léger sur le GPU
generator = pipeline(
    "text-generation",
    model=model_id,
    device=0, 
    torch_dtype=torch.float16,
    trust_remote_code=True
)
print("Modèle chargé avec succès !")

Chargement du générateur Qwen/Qwen2.5-1.5B-Instruct...


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Modèle chargé avec succès !


In [15]:
import pandas as pd
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
import os

# 1. Chargement des données (Assurez-vous d'avoir uploadé votre CSV sur Kaggle)
# Remplacez le chemin par le chemin Kaggle (ex: /kaggle/input/...)
df = pd.read_csv("/kaggle/input/politixpert-cleaned-data/politixpert_data_cleaned.csv") 

# Nettoyage
df = df[df["content"].notna()]
df = df[df["content"].str.len() > 100].reset_index(drop=True)

# 2. Fonction de découpage (Doit être IDENTIQUE à votre code local)
def chunk_text(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

# 3. Préparation
print("✂️ Découpage des textes...")
chunks_text = []
metadata_list = []

for idx, row in df.iterrows():
    text_chunks = chunk_text(row["content"])
    for i, chunk in enumerate(text_chunks):
        chunks_text.append(chunk)
        metadata_list.append({
            "title": row.get("title", "Sans titre"),
            "description": row.get("description", ""),
            "date": str(row.get("date", "")),
            "source": row["source"],
            "link": row.get("link", "#"),
            "text": chunk  # On garde le texte ici pour faciliter le chargement local
        })

print(f"📊 Total chunks: {len(chunks_text)}")

# 4. Génération des Embeddings sur GPU
print("🚀 Démarrage du modèle sur GPU...")
model = SentenceTransformer("intfloat/multilingual-e5-base", device="cuda")

print("⚡ Génération des embeddings (Cela va prendre quelques minutes)...")
embeddings = model.encode(
    chunks_text,
    batch_size=64, # Plus grand car GPU
    show_progress_bar=True,
    normalize_embeddings=True
)

# 5. Sauvegarde
print("💾 Sauvegarde des fichiers...")
np.save("embeddings.npy", embeddings)

with open("metadata.pkl", "wb") as f:
    pickle.dump(metadata_list, f)

print("✅ Terminé ! Téléchargez 'embeddings.npy' et 'metadata.pkl'")

✂️ Découpage des textes...
📊 Total chunks: 90727
🚀 Démarrage du modèle sur GPU...
⚡ Génération des embeddings (Cela va prendre quelques minutes)...


Batches:   0%|          | 0/1418 [00:00<?, ?it/s]

💾 Sauvegarde des fichiers...
✅ Terminé ! Téléchargez 'embeddings.npy' et 'metadata.pkl'


In [16]:
from collections import defaultdict

def group_results_by_party(results):
    grouped = defaultdict(list)
    for r in results:
        # On regroupe par la colonne "source" qui contient le nom du parti
        grouped[r["source"]].append(r)
    return grouped

def build_context_for_party(party, docs, max_docs=3):
    context = []
    sources = []

    for d in docs[:max_docs]:
        # 1. Récupération et nettoyage des champs
        # On utilise .get() et on gère les valeurs vides (None/NaN)
        title = d.get("title")
        if title is None: title = "بدون عنوان"
        
        description = d.get("description")
        if description is None: description = ""
        
        text_content = d["text"]

        # 2. Construction d'un BLOC COMPLET pour le modèle
        # On structure les données avec des labels clairs en arabe pour aider le modèle
        doc_entry = f"""
            معلومات الوثيقة:
            - العنوان: {title}
            - وصف السياق: {description}
            - المحتوى النصي: {text_content}
            """
        context.append(doc_entry)

        # 3. Sauvegarde pour l'affichage des sources à la fin
        sources.append({
            "title": title,
            "date": d.get("date", "Date inconnue")
        })

    # On sépare chaque document par une ligne claire
    return "\n___________________\n".join(context), sources

import re

def clean_output(text):
    # Cette expression régulière cible spécifiquement les caractères chinois (Hanzi)
    # et les remplace par une chaine vide
    text = re.sub(r'[\u4e00-\u9fff]+', '', text)
    
    # Optionnel : Nettoyer les doubles espaces créés par la suppression
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def generate_party_summary(question, party, docs):
    # 1. Préparer le contexte
    context, sources = build_context_for_party(party, docs)

    # 2. Prompt "Équilibré" (Intelligent)
    messages = [
        {"role": "system", "content": "أنت محلل سياسي خبير. مهمتك فهم السياق وتلخيص المواقف."},
        {"role": "user", "content": f"""
استناداً للوثائق التالية، لخص موقف حزب "{party}" بخصوص: "{question}".

الوثائق المتاحة (لاحظ التواريخ جيداً):
{context}

⚠️ تعليمات التلخيص:
1. **الذكاء السياقي**: إذا كان النص مؤرخاً في نهاية 2023 ويتحدث عن "مشروع قانون المالية"، فهو يقصد قانون مالية 2024. استخدم التواريخ المرفقة مع النصوص لفهم المقصود.
2. **قاعدة الاستثناء (الفضاء)**: فقط إذا كان السؤال عن "غزو الفضاء" أو "الكواكب" والنصوص سياسية، قل "لا توجد معلومات".
3. **المرونة**: لخص الموقف العام للحزب بناءً على ما هو متاح، حتى لو لم تكن الإجابة تفصيلية جداً.
4. تحدث بصيغة الغائب (يرى الحزب، يؤكد الحزب).
5. اكتب ملخصاً باللغة العربية فقط (Arabic Only).
6. لا تستخدم أي رموز غريبة أو أحرف صينية.
"""}
    ]

# ... génération ...
    try:
        output = generator(messages, max_new_tokens=300, do_sample=False, return_full_text=False)
        raw_summary = output[0]["generated_text"]
        
        # NETTOYAGE ICI
        summary_text = clean_output(raw_summary)
        
    except Exception as e:
        summary_text = "حدث خطأ."

    return {
        "party": party,
        "summary": summary_text,
        "sources": sources
    }
    
def rag_answer(question, top_k=20):
    # 1. Recherche Sémantique (Utilise votre fonction semantic_search existante)
    print(f"🔍 Recherche des documents pour : '{question}'...")
    results = semantic_search(question, top_k=top_k)
    
    # 2. Regroupement par parti
    grouped = group_results_by_party(results)
    print(f"📊 الأحزاب التي تم العثور عليها: {list(grouped.keys())}")

    # 3. Génération des résumés pour chaque parti
    answers = []
    for party, docs in grouped.items():
        print(f"🤖 جاري تلخيص موقف: {party}...")
        ans = generate_party_summary(question, party, docs)
        answers.append(ans)

    return answers

In [17]:
# 1. Définir la question
question = "ما موقف الأحزاب من قطاع الصحة؟"

# 2. Lancer le RAG
final_answers = rag_answer(question, top_k=15)

# 3. Affichage Formaté (Markdown & Emojis)
print("\n" + "="*60 + "\n")

if not final_answers:
    print("❌ لم يتم العثور على أي وثائق ذات صلة بهذا الموضوع في قاعدة البيانات.")
    print("(Le filtre de sécurité a bloqué les résultats non pertinents)")

else:
    for a in final_answers:
        summary_clean = a['summary'].strip()
        print(f"### 🏛 الحزب: {a['party']}")
        print("**📄 الخلاصة:**")
        print(summary_clean)
        print("\n**🔗 المصادر:**")
        for s in a['sources']:
            date_str = s['date'] if s['date'] else "تاريخ غير متوفر"
            print(f"- {s['title']} ({date_str})")
        print("-" * 50)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔍 Recherche des documents pour : 'ما موقف الأحزاب من قطاع الصحة؟'...
📊 الأحزاب التي تم العثور عليها: ['حزب التقدم والاشتراكية', 'الحركة الشعبية', 'الاتحاد الاشتراكي للقوات الشعبية', 'التجمع الوطني للأحرار', 'حزب الأصالة والمعاصرة']
🤖 جاري تلخيص موقف: حزب التقدم والاشتراكية...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: الحركة الشعبية...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: الاتحاد الاشتراكي للقوات الشعبية...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: التجمع الوطني للأحرار...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: حزب الأصالة والمعاصرة...


### 🏛 الحزب: حزب التقدم والاشتراكية
**📄 الخلاصة:**
**الموقف العام للحزب التقدمي والاشتراكي** - حزب التقدم والاشتراكي يدعو الحكومة إلى إتخاذ التدابير اللازمة لتحسين الخدمات الصحية العامة، خاصة في ظل تزايد أزمة الطاقة الاستيعابية. - يشير الحزب إلى أن تجاوز الطاقة الاستيعابية لمستشفيات البلاد قد يؤدي إلى تداعيات سلبية على الحياة العامة للأفراد. - يدعو الحزب إلى ضرورة تكثيف الجهود لتأمين الاحتياجات الصحية الأساسية، بما في ذلك الكشف الاختباري، توفير العدد الكافي من الأسر والوحدات العلاجية، توفير أدوات الوقاية مثل الكمامات والمعقمات، ومواكبة البروتوكول العلاجي المنزلي. - يؤكد الحزب على أهمية التضامن الوطني في مواجهة وباء كوفيد-19، وأن الحل يعتمد على توفير المزيد من الموارد والأدوات اللازمة. - يدعو الحزب إلى تغيير السياسات الحكومية وتنفيذ المشاريع الكبرى للحماية الاجتماعية والصحة، بما في ذلك تحسين الخدمات التعليمية والصحية. - يؤكد الحزب على أهمية الحوار الاجتماعي والتشريع في حل المشكلات الاقتصاديةية، وأنه يشارك في هذا الحوار من خلال دوره في البرلم

In [18]:
# 1. Définir la question
question = "ما هي مقترحات الأحزاب لإصلاح المدرسة العمومية؟"

# 2. Lancer le RAG
final_answers = rag_answer(question, top_k=15)

# 3. Affichage Formaté (Markdown & Emojis)
print("\n" + "="*60 + "\n")

if not final_answers:
    print("❌ لم يتم العثور على أي وثائق ذات صلة بهذا الموضوع في قاعدة البيانات.")
    print("(Le filtre de sécurité a bloqué les résultats non pertinents)")

else:
    for a in final_answers:
        summary_clean = a['summary'].strip()
        print(f"### 🏛 الحزب: {a['party']}")
        print("**📄 الخلاصة:**")
        print(summary_clean)
        print("\n**🔗 المصادر:**")
        for s in a['sources']:
            date_str = s['date'] if s['date'] else "تاريخ غير متوفر"
            print(f"- {s['title']} ({date_str})")
        print("-" * 50)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔍 Recherche des documents pour : 'ما هي مقترحات الأحزاب لإصلاح المدرسة العمومية؟'...
📊 الأحزاب التي تم العثور عليها: ['الاتحاد الاشتراكي للقوات الشعبية', 'حزب التقدم والاشتراكية', 'التجمع الوطني للأحرار', 'حزب الأصالة والمعاصرة', 'الحركة الشعبية']
🤖 جاري تلخيص موقف: الاتحاد الاشتراكي للقوات الشعبية...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: حزب التقدم والاشتراكية...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: التجمع الوطني للأحرار...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: حزب الأصالة والمعاصرة...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: الحركة الشعبية...


### 🏛 الحزب: الاتحاد الاشتراكي للقوات الشعبية
**📄 الخلاصة:**
**الموقف العام للاتحاد الاشتراكي للقوات الشعبية بشأن مشروع قانون المالية 2024:** - يرى الحزب أن مشروع القانون يجب أن يشمل تحسين الدخل والزيادة في الأجور والتعويضات العائلية، وتجريم الأخلاق المهنية، وتحسين الخدمات العامة، وتعزيز الحوار الاجتماعي، وتحسين التعليم العمومي. - يؤكد الحزب على أهمية التعليم العمومي كحق للشعب المغربي، وأن التعاقد مع المجتمع لا يمكن التراجع عنه. - يدعو الحزب إلى إجراءات ترشيد النفقات والحكامة الجيدة، وتحسين التعليم الأساسي وجودته، وتعزيز الديمقراطية والشفافية في الانتخابات. - يشدد الحزب على ضرورة تحقيق تمثيل حقيقي في الهياكل المنتخبة، وتجنب الانظمة الزبونية والريادية، وتعزيز الديمقراطية وشفافية العملية السياسية. - يؤكد الحزب على ضرورة تطوير الخدمات العامة وتحسين الخدمات التعليمية والصحية، وتعزيز الحوار الاجتماعي والمشاركة في القرارات المتعلقة بالتعليم والصحة.

**🔗 المصادر:**
- بلاغ المكتب السياسي لاجتماع نوفمبر (2017-11-08)
- كلمه الكاتب الاول في المجلس الوطني شتن

In [19]:
# 1. Définir la question
question = "كيف تنظر الأحزاب إلى أزمة الأساتذة المتعاقدين؟"

# 2. Lancer le RAG
final_answers = rag_answer(question, top_k=15)

# 3. Affichage Formaté (Markdown & Emojis)
print("\n" + "="*60 + "\n")

if not final_answers:
    print("❌ لم يتم العثور على أي وثائق ذات صلة بهذا الموضوع في قاعدة البيانات.")
    print("(Le filtre de sécurité a bloqué les résultats non pertinents)")

else:
    for a in final_answers:
        summary_clean = a['summary'].strip()
        print(f"### 🏛 الحزب: {a['party']}")
        print("**📄 الخلاصة:**")
        print(summary_clean)
        print("\n**🔗 المصادر:**")
        for s in a['sources']:
            date_str = s['date'] if s['date'] else "تاريخ غير متوفر"
            print(f"- {s['title']} ({date_str})")
        print("-" * 50)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔍 Recherche des documents pour : 'كيف تنظر الأحزاب إلى أزمة الأساتذة المتعاقدين؟'...
📊 الأحزاب التي تم العثور عليها: ['حزب التقدم والاشتراكية', 'التجمع الوطني للأحرار', 'حزب الأصالة والمعاصرة', 'الاتحاد الاشتراكي للقوات الشعبية']
🤖 جاري تلخيص موقف: حزب التقدم والاشتراكية...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: التجمع الوطني للأحرار...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: حزب الأصالة والمعاصرة...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: الاتحاد الاشتراكي للقوات الشعبية...


### 🏛 الحزب: حزب التقدم والاشتراكية
**📄 الخلاصة:**
**الموقف العام للحزب التقدمي والاشتراكي** - حزب التقدم والاشتراكي يرى أن نجاح أي إصلاح للمنظومة التربوية يعتمد على المشاركة الكاملة والكاملة للأعضاء التعليميين، خاصة الأساتذة المتعاقدين. - حزب التقدم والاشتراكي يؤكد على الموقف الثابت والواضح للحزب، وهو التساوي في الحقوق بين جميع مكونات الأسرة التعليمية، بما في ذلك الأساتذة المتعاقدين. - حزب التقدم والاشتراكي يشير إلى أنه عقد لقاءات مع المهنية التربوية والتكوين، بما في ذلك الأساتذة المتعاقدين، وكان آخرها الاستقبال الذي خص به الأمين العام للحزب الأساتذة المنضوين تحت لواء التنسيقية التي فرضتهم التعاقد. - حزب التقدم والاشتراكي يؤكد على عدم وجود تضامن أو تعاون حكوميا مع أساتذة المتعاقدين، وأنه لا يوجد اعتبار لواقعهم أو حقوقهم. - حزب التقدم والاشتراكي يدعو الحكومة إلى العمل على تجاوز الأزمة وإعادة النظر في القضايا المتعلقة بالأأساتذة المتعاقدين، بما في ذلك إعادة النظر في التعديلات التي تم طرحها في القانون المالي.

**🔗 المصادر:**
- محمد

In [20]:
# 1. Définir la question
question = "كيف تقيم الأحزاب قانون المالية لسنة 2024؟"

# 2. Lancer le RAG
final_answers = rag_answer(question, top_k=40)

# 3. Affichage Formaté (Markdown & Emojis)
print("\n" + "="*60 + "\n")

if not final_answers:
    print("❌ لم يتم العثور على أي وثائق ذات صلة بهذا الموضوع في قاعدة البيانات.")
    print("(Le filtre de sécurité a bloqué les résultats non pertinents)")

else:
    for a in final_answers:
        summary_clean = a['summary'].strip()
        print(f"### 🏛 الحزب: {a['party']}")
        print("**📄 الخلاصة:**")
        print(summary_clean)
        print("\n**🔗 المصادر:**")
        for s in a['sources']:
            date_str = s['date'] if s['date'] else "تاريخ غير متوفر"
            print(f"- {s['title']} ({date_str})")
        print("-" * 50)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔍 Recherche des documents pour : 'كيف تقيم الأحزاب قانون المالية لسنة 2024؟'...
📊 الأحزاب التي تم العثور عليها: ['الاتحاد الاشتراكي للقوات الشعبية', 'حزب التقدم والاشتراكية', 'التجمع الوطني للأحرار', 'حزب الأصالة والمعاصرة']
🤖 جاري تلخيص موقف: الاتحاد الاشتراكي للقوات الشعبية...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: حزب التقدم والاشتراكية...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: التجمع الوطني للأحرار...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: حزب الأصالة والمعاصرة...


### 🏛 الحزب: الاتحاد الاشتراكي للقوات الشعبية
**📄 الخلاصة:**
**الموقف العام للاتحاد الاشتراكي للقوات الشعبية بشأن قانون المالية لسنة 2024:** - يرى الحزب أن قانون المالية لسنة 2024 خارج القانون التنظيمي والدستوري. - يعتقد أن هذا القانون يشكل تحدياً للحرية السياسية للأحزاب. - يدعو إلى تقييم الأداء الاقتصادي والاجتماعي الحالي، حيث يرون أنه قد استنفذ جميع إمكاناته. - يشير إلى الحاجة إلى تطوير نموذج اقتصادي وتنموي جديد. - يؤكد على أهمية الحوار والتعاون بين الحكومة والمعارضة. - يشجع على تنظيم ندوات حول محاور تهم مختلف الاشكالات المطروحة. - يطالب بتوضيح الاكراهات الاقتصادية والاجتماعية المحيطة بالوضع الاقتصادي المغربي. - يؤكد على ضرورة تحسين مناخ الاستثمار وتحسين الخدمات الاجتماعية. - يدعو إلى تحسين نظام الضريبة وتحسين مناخ الأعمال.

**🔗 المصادر:**
- حتي لا تتحول مجالس الحكامه الي اليه للتحكم (2024-03-14)
- اليوم الدراسي للفريقين الاشتراكيين بالبرلمان حول قانون الماليه للسنه المقبله (2016-08-26)
- في بلاغ المكتب السياسي من اجل قانون للماليه اجتماع

In [21]:
# 1. Définir la question
question = "ما هو موقف الأحزاب من غزو الفضاء؟"

# 2. Lancer le RAG
final_answers = rag_answer(question, top_k=15)

# 3. Affichage Formaté (Markdown & Emojis)
print("\n" + "="*60 + "\n")

if not final_answers:
    print("❌ لم يتم العثور على أي وثائق ذات صلة بهذا الموضوع في قاعدة البيانات.")
    print("(Le filtre de sécurité a bloqué les résultats non pertinents)")

else:
    for a in final_answers:
        summary_clean = a['summary'].strip()
        print(f"### 🏛 الحزب: {a['party']}")
        print("**📄 الخلاصة:**")
        print(summary_clean)
        print("\n**🔗 المصادر:**")
        for s in a['sources']:
            date_str = s['date'] if s['date'] else "تاريخ غير متوفر"
            print(f"- {s['title']} ({date_str})")
        print("-" * 50)

🔍 Recherche des documents pour : 'ما هو موقف الأحزاب من غزو الفضاء؟'...
📊 الأحزاب التي تم العثور عليها: []


❌ لم يتم العثور على أي وثائق ذات صلة بهذا الموضوع في قاعدة البيانات.
(Le filtre de sécurité a bloqué les résultats non pertinents)


In [22]:
# 1. Définir la question
question = " ما رأي حزب الاستقلال في الرقمنة؟"

# 2. Lancer le RAG
final_answers = rag_answer(question, top_k=15)

# 3. Affichage Formaté (Markdown & Emojis)
print("\n" + "="*60 + "\n")

if not final_answers:
    print("❌ لم يتم العثور على أي وثائق ذات صلة بهذا الموضوع في قاعدة البيانات.")
    print("(Le filtre de sécurité a bloqué les résultats non pertinents)")

else:
    for a in final_answers:
        summary_clean = a['summary'].strip()
        print(f"### 🏛 الحزب: {a['party']}")
        print("**📄 الخلاصة:**")
        print(summary_clean)
        print("\n**🔗 المصادر:**")
        for s in a['sources']:
            date_str = s['date'] if s['date'] else "تاريخ غير متوفر"
            print(f"- {s['title']} ({date_str})")
        print("-" * 50)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔍 Recherche des documents pour : ' ما رأي حزب الاستقلال في الرقمنة؟'...
📊 الأحزاب التي تم العثور عليها: ['التجمع الوطني للأحرار', 'حزب الأصالة والمعاصرة', 'الاتحاد الاشتراكي للقوات الشعبية']
🤖 جاري تلخيص موقف: التجمع الوطني للأحرار...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: حزب الأصالة والمعاصرة...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🤖 جاري تلخيص موقف: الاتحاد الاشتراكي للقوات الشعبية...


### 🏛 الحزب: التجمع الوطني للأحرار
**📄 الخلاصة:**
**الموقف العام للحزب التجمع الوطني للأحرار بشأن الرقمنة:** - يقترح الحزب تسريع وتيرة التحول الرقمي في المغرب، خاصة في مجال التعليم والصحة. - يدعو إلى إنشاء مخططات رقمية في هذه المجالات لدعم التعلم عن بعد وتطبيق التحول الرقمي في الإدارة العامة. - يؤكد على ضرورة تعزيز مراقبة جودة الخدمات العمومية وتعزيز المراقبة في القطاعات الاجتماعية. - يدعو إلى إنشاء هيئات مستقلة لقياس نجاعة السياسات العمومية في الصحة والتعليم. يرى الحزب أن هذا التوجه يساهم في تمكين التلاميذ من الدراسة عن بعد وتحسين التحول الرقمي في الإدارة العامة، مما يؤدي إلى تحسين جودة الخدمات العمومية.

**🔗 المصادر:**
- الاحرار يقترح احداث صندوق لتفعيل الطابع الرسمي للامازيغيه بموارد ماليه تصل الي مليار درهم في افق (2021-06-07)
--------------------------------------------------
### 🏛 الحزب: حزب الأصالة والمعاصرة
**📄 الخلاصة:**
**الموقف العام للحزب "حزب الأصالة والمعاصرة":** - حزب الأصالة والمعاصرة يرى أن الرقمنة ستكون مصدراً م